In [2]:
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.ddpg import DDPGTrainer
import ray
import shared_constants
import tensorflow as tf
from ray.rllib.contrib.maddpg import MADDPGTrainer
from ray.tune import register_env
from gym_pybullet_drones.envs.multi_agent_rl.PayloadCoop import PayloadCoop
from ray.rllib.agents.callbacks import DefaultCallbacks
import shared_constants
from gym_pybullet_drones.envs.single_agent_rl.BaseSingleAgentAviary import ActionType, ObservationType

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
current_phase=0
class FillInActions(DefaultCallbacks):
    def on_train_result(self, trainer, result):
        global current_phase
        print("Manage Curriculum callback called on phase {}".format(current_phase))
        if result["episode_reward_mean"] > 0:
            current_phase+=1
            trainer.workers.foreach_worker(
            lambda ev: ev.foreach_env(
                lambda env: env.set_phase(current_phase)))

In [111]:
import shared_constants
from gym_pybullet_drones.envs.single_agent_rl.BaseSingleAgentAviary import ActionType, ObservationType

ray.shutdown()
ray.init()
NUM_DRONES =shared_constants.NUM_DRONES
ACT = ActionType(shared_constants.ACT)
OBS = ObservationType(shared_constants.OBS)
temp_env_name = "this-aviary-v0" 
register_env(temp_env_name, lambda _: PayloadCoop(num_drones=NUM_DRONES,
                                                    aggregate_phy_steps=shared_constants.AGGR_PHY_STEPS,
                                                    obs=OBS,
                                                    act=ACT
                                                    ))
temp_env = PayloadCoop(num_drones=NUM_DRONES,
                            aggregate_phy_steps=shared_constants.AGGR_PHY_STEPS,
                            obs=OBS,
                            act=ACT,
                            )
action_space = temp_env.action_space[0]
observation_space = temp_env.observation_space[0]

obs_space_dict = {
    "agent_1": observation_space,
    "agent_2": observation_space,
}
act_space_dict = {
    "agent_1": action_space,
    "agent_2": action_space,
}

#### Config ##################################################
config = {
    "env": temp_env_name,
    "env_config": {
        "actions_are_logits": False,
    },
    "multiagent": {
        "policies": {
            "pol1": (None, observation_space, action_space, {
                "agent_id": 0,
            }),
            "pol2": (None, observation_space, action_space, {
                "agent_id": 1,
            }),
        },
        "policy_mapping_fn": lambda x: "pol1" if x == 0 else "pol2",
    },
    "framework": "tf",
    "callbacks":FillInActions,
    "gamma": shared_constants.GAMMA,
    "n_step": shared_constants.N_STEP,
    "num_workers": 0 + 4,
    "num_envs_per_worker": 1,
    "batch_mode": "complete_episodes",

    "actor_hiddens": [64, 64],
    "critic_hiddens": [64, 64],
    "actor_lr": shared_constants.LR,
    "critic_lr": shared_constants.LR,
    "tau": 0.01,
    "train_batch_size": 512,
    "learning_starts": 512 * 10,
    "explore": False,
    'exploration_config': {"type": "OrnsteinUhlenbeckNoise"},
}

2021-03-24 20:06:56,278	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8265


[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 0.000000, km 0.000000,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000


/home/mahendra/anaconda3/envs/TA/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [112]:
tr = MADDPGTrainer(config = config)

(pid=15647) WARNING:tensorflow:From /home/mahendra/anaconda3/envs/TA/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=15647) Instructions for updating:
(pid=15647) non-resource variables are not supported in the long term
(pid=15649) WARNING:tensorflow:From /home/mahendra/anaconda3/envs/TA/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=15649) Instructions for updating:
(pid=15649) non-resource variables are not supported in the long term
(pid=15646) WARNING:tensorflow:From /home/mahendra/anaconda3/envs/TA/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a fut

(pid=15649) [INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
(pid=15649) [INFO] m 0.027000, L 0.039700,
(pid=15649) [INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
(pid=15649) [INFO] kf 0.000000, km 0.000000,
(pid=15649) [INFO] t2w 2.250000, max_speed_kmh 30.000000,
(pid=15649) [INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
(pid=15649) [INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
(pid=15649) [INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000
(pid=15646) [INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
(pid=15646) [INFO] m 0.027000, L 0.039700,
(pid=15646) [INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
(pid=15646) [INFO] kf 0.000000, km 0.000000,
(pid=15646) [INFO] t2w 2.250000, max_speed_kmh 30.000000,
(pid=15646) [INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
(pid=15646) [INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
(pid=15646) [INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coef

(pid=15642) WARNING:tensorflow:From /home/mahendra/anaconda3/envs/TA/lib/python3.6/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:187: Layer.apply (from tensorflow.python.keras.engine.base_layer_v1) is deprecated and will be removed in a future version.
(pid=15642) Instructions for updating:
(pid=15642) Please use `layer.__call__` method instead.
2021-03-24 20:07:37,895	INFO trainable.py:102 -- Trainable.setup took 29.060 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2021-03-24 20:07:37,896	WARNING util.py:43 -- Install gputil for GPU system monitoring.


In [99]:
pl1 = tr.get_policy("pol1")

In [100]:
a = observation_space.sample().reshape(1, -1)

In [108]:
pl1.compute_actions(a)

(array([[0.01323674, 0.2584071 , 0.24060018, 0.4355373 , 0.05221861]],
       dtype=float32),
 [],
 {'action_dist_inputs': array([[-0.22037807,  0.06384001, -0.1733896 ,  0.24483114,  0.10656784]],
        dtype=float32)})

In [113]:
pl1.config

{'num_workers': 4,
 'num_envs_per_worker': 1,
 'create_env_on_driver': False,
 'rollout_fragment_length': 100,
 'batch_mode': 'complete_episodes',
 'num_gpus': 0,
 'train_batch_size': 512,
 'model': {'fcnet_hiddens': [256, 256],
  'fcnet_activation': 'tanh',
  'conv_filters': None,
  'conv_activation': 'relu',
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': True,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action': False,
  'lstm_use_prev_reward': False,
  '_time_major': False,
  'framestack': True,
  'dim': 84,
  'grayscale': False,
  'zero_mean': True,
  'custom_model': None,
  'custom_model_config': {},
  'custom_action_dist': None,
  'custom_preprocessor': None,
  'lstm_use_prev_action_reward': -1},
 'optimizer': {},
 'gamma': 0.99,
 'horizon': None,
 'soft_horizon': False,
 'no_done_at_end': False,
 'env_config': {'actions_are_logits': False},
 'env': 'this-aviary-v0',
 'normalize_actions': False,
 'clip_rewards': N

In [97]:
tr.config

{'num_workers': 4,
 'num_envs_per_worker': 1,
 'create_env_on_driver': False,
 'rollout_fragment_length': 100,
 'batch_mode': 'complete_episodes',
 'num_gpus': 0,
 'train_batch_size': 512,
 'model': {'fcnet_hiddens': [256, 256],
  'fcnet_activation': 'tanh',
  'conv_filters': None,
  'conv_activation': 'relu',
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': True,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action': False,
  'lstm_use_prev_reward': False,
  '_time_major': False,
  'framestack': True,
  'dim': 84,
  'grayscale': False,
  'zero_mean': True,
  'custom_model': None,
  'custom_model_config': {},
  'custom_action_dist': None,
  'custom_preprocessor': None,
  'lstm_use_prev_action_reward': -1},
 'optimizer': {},
 'gamma': 0.99,
 'horizon': None,
 'soft_horizon': False,
 'no_done_at_end': False,
 'env_config': {'actions_are_logits': False},
 'env': 'this-aviary-v0',
 'normalize_actions': False,
 'clip_rewards': N